In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
_ = """
Function decorators let us “mark” functions in the source code to enhance their behavior
in some way.
"""
# decorator 101
_ = """
A decorator is a callable that takes another function as argument (the decorated function)
The decorator may perform some processing with the decorated function, and
returns it or replaces it with another function or callable object.

To summarize: the first crucial fact about decorators is that they have the power to
replace the decorated function with a different one. The second crucial fact is that they
are executed immediately when a module is loaded. This is explained next
"""
# @decorate
# def target():
#     print('running target()')
# has the same effect
# def target():
#     print('running target()')
# target = decorate(target)

def deco(func):
    def inner():
        print('running inner()')
    return inner


@deco
def target():
    print('running target()')


target()

running inner()


In [5]:
# when python executes decorators
_ = """
A key feature of decorators is that they run right after the decorated function is defined
"""
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func
@register
def f1():
    print('running f1()')
@register
def f2():
    print('running f2()')
def f3():
    print('running f3()')
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
main()

running register(<function f1 at 0x0000025A92CD8378>)
running register(<function f2 at 0x0000025A92CD8C80>)
running main()
registry -> [<function f1 at 0x0000025A92CD8378>, <function f2 at 0x0000025A92CD8C80>]
running f1()
running f2()
running f3()


In [6]:
# decorator-enhanced strategy pattern
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order_promo(order):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)




In [5]:
def f1(a):
    print(a)
    print(b)


b = 3
f1(3)


d = 3
def f2(c):
    print(c)
    print(d)
    d = 9


f2(3)
_ = """
But the fact is, when Python compiles the body of the function, it decides that b is a local
variable because it is assigned within the function. The generated bytecode reflects this
decision and will try to fetch b from the local environment. Later, when the call f2(3)
is made, the body of f2 fetches and prints the value of the local variable a, but when
trying to fetch the value of local variable b it discovers that b is unbound.
"""

3
3
3


UnboundLocalError: local variable 'd' referenced before assignment

In [7]:
def f3(a):
    global b
    print(a)
    print(b)
    b = 9


f3(3)
b

3
9


9

In [8]:
# closures
_ = """
Actually, a closure is a function with an extended scope that encompasses nonglobal
variables referenced in the body of the function but not defined there. It does not matter
whether the function is anonymous or not; what matters is that it can access nonglobal
variables that are defined outside of its body.
"""
# an example
_ = """
Consider an avg function to compute the mean of an ever-increasing series of values;
for example, the average closing price of a commodity over its entire history. Every day
a new price is added, and the average is computed taking into account all prices so far.
"""
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)


avg = Averager()
avg(10)
avg(11)
avg(12)


def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager


avg = make_averager()
avg(10)
avg(11)
avg(12)


10.0

10.5

11.0

10.0

10.5

11.0

In [9]:
# the nonlocal declaration
_ = """
But with immutable types like numbers, strings, tuples, etc., all you can do is read, but
never update. If you try to rebind them, as in count = count + 1, then you are implicitly
creating a local variable count. It is no longer a free variable, and therefore it is not saved
in the closure.
To work around this, the nonlocal declaration was introduced in Python 3
"""
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager


avg = make_averager()
avg(10)
avg(11)
avg(12)

10.0

10.5

11.0

In [10]:
# a simple decorator
import time
def clock(func):
    def clocked(*args): #
        t0 = time.perf_counter()
        result = func(*args) #
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


import time
@clock
def snooze(seconds):
    time.sleep(seconds)


@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)


print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12360310s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000090s] factorial(1) -> 1
[0.00035530s] factorial(2) -> 2
[0.00045650s] factorial(3) -> 6
[0.00051980s] factorial(4) -> 24
[0.00055000s] factorial(5) -> 120
[0.00060890s] factorial(6) -> 720
6! = 720
